In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
import numpy as np
from pathlib import Path

# Functions

In [2]:
def weighted_mean(x, metric: str):
    if len(x[metric].dropna()) == 0:
        res = 0
    else:
        res = (x[metric] * x["sample_weight"]).sum()/x["sample_weight"].sum()
    col_name = f"{metric} wt_mean"
    return pd.Series({col_name: res})

def weighted_percentile(x, metric: str, percents: int):
    col_name = f"{metric} wt_p{percents}"
    res = _weighted_percentile(x[metric], percents, weights=x["sample_weight"])
    return pd.Series({col_name: res})

def _weighted_percentile(data, percents: int, weights=None):
    """
    perc : percentile in 0 - 1
    """
    # dropna
    idx = ~np.isnan(data) & ~np.isnan(weights)
    data = np.array(data)[idx]
    if len(data) == 0:
        return np.nan
    if weights is None:
        return np.percentile(data, percents)
    weights = np.array(weights)[idx]
    ix = np.argsort(data)
    data = data[ix] # sort data
    weights = weights[ix] # sort weights
    cdf = (np.cumsum(weights) - 0.5 * weights) / np.sum(weights) # 'like' a CDF function
    return np.interp(percents, cdf, data)

def get_summary(df, metric, lower_percentile=0.25, upper_percentile=0.75):
    dfgb = df.groupby(["upgrade_name"])
    dfs = pd.concat([
        dfgb["sample_weight"].count().rename("n_samples"),
        dfgb["sample_weight"].sum().rename("n_represented"),
        dfgb.apply(lambda x: weighted_mean(x, metric)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=lower_percentile)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=upper_percentile)),
    ], axis=1)
    dfs = dfs.sort_values(by=f"{metric} wt_mean", ascending=False)

    return dfs

## Whole Community Upgrade Cost Per Dwelling Unit

In [3]:
def community_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']/results['sample_weight']
    results = results.sort_values('upgrade_cost_usd')
    results['upgrade_cost_usd'] = results['upgrade_cost_usd'].round(1) 
    
    print('upgrade_cost_usd per dwelling unit')
    display(results['upgrade_cost_usd'])
    return results['upgrade_cost_usd']

## Whole Community Total Annual Savings

In [4]:
# flavor 1 - with electricity and non-electricity split
def figure_total_savings_1(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.electricity_kwh',
                      'saving_energy.natural_gas_therm',
                                'saving_energy.fuel_oil_mmbtu',
                                'saving_energy.propane_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=5, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=5)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission<br>Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Electricity<br>Savings<br>kwh", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Gas, Propane,<br>Fuel Oil Savings<br>mmbtu", xaxis5_visible=True, yaxis5_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=900,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_1.svg')    

# flavor 2 - site, electricity, and non-electricity
def figure_total_savings_2(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_energy.electricity_kwh',
        'saving_energy.natural_gas_therm',
        'saving_energy.fuel_oil_mmbtu',
        'saving_energy.propane_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                      'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=6, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)

    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=5)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=6)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=6)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Site Emission<br>Saving<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Site Energy<br>Saving<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Electricity<br>Saving<br>kwh", xaxis5_visible=True, yaxis5_visible=False,
                      xaxis6_title="Gas, Propane,<br>Fuel Oil Saving<br>mmbtu", xaxis6_visible=True, yaxis6_visible=False,
                     title='Community Total Annual Savings')
    fig.update_layout(
    autosize=False,
    width=1100,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_2.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu','saving_energy.electricity_kwh','saving_energy.natural_gas_fuel_oil_propane_mmbtu']]

# flavor 3 - site energy only
def figure_total_savings_3(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_3.svg')

## Whole Community Annual Savings Per Dewlling Unit

In [5]:
def figure_mean_savings(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.total_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']/results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']/results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']/results['sample_weight']
    results = results.sort_values('saving_cbill.total_usd')
    
    print('sample_weight')
    display(results['sample_weight'])
  
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Annual Savings Per Dwelling Unit')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'annual_saving_per_dwelling_unit.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu']]

## Segment Upgrade Cost Per Dwelling Unit

In [6]:
def segment_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight',
                                'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    segment5 = results.loc[results['segment'] == 'segment5']
    
    agg_functions = {'upgrade_name': 'first',
                     'segment': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    
    segment1 = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2 = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3 = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4 = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    segment5 = segment5.groupby(segment5['upgrade_name']).aggregate(agg_functions)
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd']/segment1['sample_weight']
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd']/segment2['sample_weight']
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd']/segment3['sample_weight']
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd']/segment4['sample_weight']
    segment5['upgrade_cost_usd'] = segment5['upgrade_cost_usd']/segment5['sample_weight']
    
    segment1 = segment1.sort_values('upgrade_cost_usd')
    segment2 = segment2.sort_values('upgrade_cost_usd')
    segment3 = segment3.sort_values('upgrade_cost_usd')
    segment4 = segment4.sort_values('upgrade_cost_usd')
    segment5 = segment5.sort_values('upgrade_cost_usd')
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd'].round(1)
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd'].round(1)
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd'].round(1)
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd'].round(1)
    segment5['upgrade_cost_usd'] = segment5['upgrade_cost_usd'].round(1)
    
    print('segment1: upgrade cost (usd) per dwelling unit')
    display(segment1['upgrade_cost_usd'])
    print('segment2: upgrade cost (usd) per dwelling unit')
    display(segment2['upgrade_cost_usd'])
    print('segment3: upgrade cost (usd) per dwelling unit')
    display(segment3['upgrade_cost_usd'])
    print('segment4: upgrade cost (usd) per dwelling unit')
    display(segment4['upgrade_cost_usd'])
    print('segment5: upgrade cost (usd) per dwelling unit')
    display(segment5['upgrade_cost_usd'])
    return (segment1['upgrade_cost_usd'], segment2['upgrade_cost_usd'], segment3['upgrade_cost_usd'], segment4['upgrade_cost_usd'], segment5['upgrade_cost_usd'])

## Segment Savings Per Dollar Invested

In [7]:
def segment_savings_per_dollar(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'upgrade_cost_usd', 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    segment5 = results.loc[results['segment'] == 'segment5']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'upgrade_cost_usd': 'sum',
                     'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    segment5_agg = segment5.groupby(segment5['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['upgrade_cost_usd']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['upgrade_cost_usd']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['upgrade_cost_usd']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['upgrade_cost_usd']
    segment5_agg[indicator] = segment5_agg[indicator]/segment5_agg['upgrade_cost_usd']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    segment5_agg = segment5_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(5)
    segment2_agg[indicator] = segment2_agg[indicator].round(5)
    segment3_agg[indicator] = segment3_agg[indicator].round(5)
    segment4_agg[indicator] = segment4_agg[indicator].round(5)
    segment5_agg[indicator] = segment5_agg[indicator].round(5)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator]) 
    print('segment5')
    display(segment5_agg[indicator]) 
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator],segment5_agg[indicator])

## Segment Annual Total Savings

In [8]:
def segment_annual_total_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    segment5 = results.loc[results['segment'] == 'segment5']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    segment5_agg = segment5.groupby(segment5['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    segment5_agg = segment5_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)
    segment5_agg[indicator] = segment5_agg[indicator].round(1)

    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator])
    print('segment5')
    display(segment5_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator],segment5_agg[indicator])

## Segment Annual Savings Per Dwelling Unit

In [9]:
def segment_mean_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    segment5 = results.loc[results['segment'] == 'segment5']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    segment5_agg = segment5.groupby(segment5['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    segment5_agg[indicator] = segment5_agg[indicator]/segment5_agg['sample_weight']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    segment5_agg = segment5_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)
    segment5_agg[indicator] = segment5_agg[indicator].round(1)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator])
    print('segment5')
    display(segment5_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator],segment5_agg[indicator])

## Segment Percent Stock with Net Positive Savings

In [10]:
def figure_positive_negative_savings(segment, commnity_results):
    results = commnity_results[['upgrade_name',
                                  'saving_energy.total_mmbtu',
                                  'saving_cbill.total_usd',
                                  'saving_emission.total_kgCO2e',
                                'sample_weight',
                               'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results = results.loc[results['segment'] == segment]
    results = results.groupby(results['upgrade_name'])
  
    def pos(col): 
      return col[col > 0].count()
  
    def neg(col): 
      return col[col < 0].count() 

    percentage = pd.DataFrame()

    bill = results['saving_cbill.total_usd'].agg([('negative_values', neg),('positive_values', pos)])
    emission = results['saving_emission.total_kgCO2e'].agg([('negative_values', neg),('positive_values', pos)])
    energy = results['saving_energy.total_mmbtu'].agg([('negative_values', neg),('positive_values', pos)])
    
    percentage ['cbill_n'] = bill['negative_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['cbill_p'] = bill['positive_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['emission_n'] = emission['negative_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['emission_p'] = emission['positive_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['energy_n'] = energy['negative_values']/(energy['negative_values']+energy['positive_values'])
    percentage ['energy_p'] = energy['positive_values']/(energy['negative_values']+energy['positive_values'])
    
    
    percentage = percentage.sort_values(('cbill_p'))
    upgrade_name = percentage.index
    
    fig = make_subplots(rows=1, cols=3)
    marker1= dict(color='red')
    marker2= dict(color='green')
    
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_n'], text=percentage ['cbill_n'],
                orientation='h',name = "Negative",row=1, col=1, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_p'],text=percentage ['cbill_p'],
                orientation='h', name = "Positive", row=1, col=1, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['emission_n'], text=percentage ['emission_n'],
                orientation='h',showlegend=False, row=1, col=2, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['emission_p'],text=percentage ['emission_p'],
                orientation='h', showlegend=False, row=1, col=2, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['energy_n'],text=percentage ['energy_n'],
                orientation='h',showlegend=False, row=1, col=3, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['energy_p'],text=percentage ['energy_p'],
                orientation='h', showlegend=False, row=1, col=3, marker=marker2)
   
  
    fig.update_layout(barmode="relative")
    fig.update_traces(texttemplate='%{text:.0%}', textposition='inside')
    
    fig.update_layout(xaxis_title="Bill", xaxis_visible=True, yaxis_visible=True,
                      xaxis2_title="Emission", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Total Energy", xaxis3_visible=True, yaxis3_visible=False,
                      font=dict(size=10))
    
    fig.update_layout(
        title='Percentage of Household in Positive and Negative Savings')
    
    fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    font=dict(size=10))
    fig.layout.xaxis1.tickformat = ',.0%'
    fig.layout.xaxis2.tickformat = ',.0%'
    fig.layout.xaxis3.tickformat = ',.0%'
    
    fig.show()
    fig.write_image(fig_dir / f'percentage_positive_saving_{segment}.svg')
    return (percentage ['cbill_p'],percentage ['emission_p'],percentage ['energy_p'])

## Segment Average Energy Burden (%) Before and after Upgrade.

In [11]:
def segment_mean_energy_burden(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'baseline_energy_burden_2023_cbills.%', 'sample_weight', 'segment']]
    results.replace([np.inf, -np.inf], np.nan, inplace=True)
    results.dropna(subset=['baseline_energy_burden_2023_cbills.%'], how="all", inplace=True)
    results[indicator] = results[indicator]*results['sample_weight']
    results['baseline_energy_burden_2023_cbills.%'] = results['baseline_energy_burden_2023_cbills.%']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    segment5 = results.loc[results['segment'] == 'segment5']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'baseline_energy_burden_2023_cbills.%': 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    segment5_agg = segment5.groupby(segment5['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg['baseline_energy_burden_2023_cbills.%'] = segment1_agg['baseline_energy_burden_2023_cbills.%']/segment1_agg['sample_weight']
    segment2_agg['baseline_energy_burden_2023_cbills.%'] = segment2_agg['baseline_energy_burden_2023_cbills.%']/segment2_agg['sample_weight']
    segment3_agg['baseline_energy_burden_2023_cbills.%'] = segment3_agg['baseline_energy_burden_2023_cbills.%']/segment3_agg['sample_weight']
    segment4_agg['baseline_energy_burden_2023_cbills.%'] = segment4_agg['baseline_energy_burden_2023_cbills.%']/segment4_agg['sample_weight']
    segment5_agg['baseline_energy_burden_2023_cbills.%'] = segment5_agg['baseline_energy_burden_2023_cbills.%']/segment5_agg['sample_weight']
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    segment5_agg[indicator] = segment5_agg[indicator]/segment5_agg['sample_weight']
    
    segment1_agg['energy_burden_diff'] = segment1_agg['baseline_energy_burden_2023_cbills.%'] - segment1_agg[indicator]
    segment2_agg['energy_burden_diff'] = segment2_agg['baseline_energy_burden_2023_cbills.%'] - segment2_agg[indicator]
    segment3_agg['energy_burden_diff'] = segment3_agg['baseline_energy_burden_2023_cbills.%'] - segment3_agg[indicator]
    segment4_agg['energy_burden_diff'] = segment4_agg['baseline_energy_burden_2023_cbills.%'] - segment4_agg[indicator]
    segment5_agg['energy_burden_diff'] = segment5_agg['baseline_energy_burden_2023_cbills.%'] - segment5_agg[indicator]
    
    segment1_agg = segment1_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment2_agg = segment2_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment3_agg = segment3_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment4_agg = segment4_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment5_agg = segment5_agg.sort_values(['energy_burden_diff'], ascending=False)
    
    segment1_agg = segment1_agg[segment1_agg.upgrade_name != 'baseline']
    segment2_agg = segment2_agg[segment2_agg.upgrade_name != 'baseline']
    segment3_agg = segment3_agg[segment3_agg.upgrade_name != 'baseline']
    segment4_agg = segment4_agg[segment4_agg.upgrade_name != 'baseline']
    segment5_agg = segment5_agg[segment5_agg.upgrade_name != 'baseline']
    
    segment1_agg = segment1_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment2_agg = segment2_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment3_agg = segment3_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment4_agg = segment4_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment5_agg = segment5_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    
    segment1_agg['before'] = segment1_agg['before'].round(1)
    segment2_agg['before'] = segment2_agg['before'].round(1)
    segment3_agg['before'] = segment3_agg['before'].round(1)
    segment4_agg['before'] = segment4_agg['before'].round(1)
    segment5_agg['before'] = segment5_agg['before'].round(1)
    segment1_agg['after'] = segment1_agg['after'].round(1)
    segment2_agg['after'] = segment2_agg['after'].round(1)
    segment3_agg['after'] = segment3_agg['after'].round(1)
    segment4_agg['after'] = segment4_agg['after'].round(1)
    segment5_agg['after'] = segment5_agg['after'].round(1)
    
    print ('energy burden (%) before and after upgrade')
    print('segment1')
    display(segment1_agg[['before', 'after']])
    print('segment2')
    display(segment2_agg[['before', 'after']])
    print('segment3')
    display(segment3_agg[['before', 'after']])
    print('segment4')
    display(segment4_agg[['before', 'after']])
    print('segment5')
    display(segment5_agg[['before', 'after']])
    
    return(segment1_agg[['before', 'after']],segment2_agg[['before', 'after']],segment3_agg[['before', 'after']],segment4_agg[['before', 'after']],segment5_agg[['before', 'after']])

In [12]:
community = 'duluth'
fig_dir = Path(".").resolve() / "results" / community
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")
saving_results = pd.DataFrame()

# 1. Results for Whole Community

## 1.1 Upgrade Cost Per Dwelling Unit

In [13]:
results = community_mean_upgrade_cost(community_results)
saving_results['whole_community_upgrade_cost_mean_usd'] = results

upgrade_cost_usd per dwelling unit


upgrade_name
Electric Cooking                                                    1127.3
Electric Clothes Dryer                                              1266.0
Induction Cooking                                                   1953.1
Heat Pump Clothes Dryer                                             2547.5
Basic Enclosure                                                     2940.3
Heat Pump Water Heater                                              3194.7
Enhanced Enclosure                                                  4934.1
High Efficiency Heat Pump with Electric Heat Backup                31202.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        34316.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     34669.7
Basic Enclosure + High Efficiency Whole Home Electrification       38326.7
High Efficiency Whole Home Electrification                         38404.3
Enhanced Enclosure + High Efficiency Whole Home Electrification    38679.8
Mininum Effi

## 1.2 Total Annual Savings

In [14]:
figure_total_savings_1(community_results)

In [15]:
results = figure_total_savings_2(community_results)
saving_results[['whole_community_saving_bill_total_usd','whole_community_saving_emission_total_kgCO2e','whole_community_saving_energy_total_mmbtu','whole_community_saving_electricity_total_kwh','whole_community_saving_natural_gas_fuel_oil_propane_total_mmbtu']] = results

In [16]:
figure_total_savings_3(community_results) 

## 1.3 Annual Savings Per Dwelling Unit

In [17]:
results = figure_mean_savings(community_results) 
saving_results[['whole_community_saving_bill_mean_usd','whole_community_saving_emission_mean_kgCO2e','whole_community_saving_energy_mean_mmbtu']] = results

sample_weight


upgrade_name
Mininum Efficiency Whole Home Electrification                      35144.658339
Mininum Efficiency Heat Pump with Electric Heat Backup             39762.000000
Electric Clothes Dryer                                              5914.743706
High Efficiency Heat Pump with Electric Heat Backup                39762.000000
Electric Cooking                                                   12722.230006
Induction Cooking                                                  39329.804348
Heat Pump Clothes Dryer                                            32436.993494
Mininum Efficiency Heat Pump with Existing Heat Backup             37835.613214
Heat Pump Water Heater                                             39724.904866
High Efficiency Whole Home Electrification                         39762.000000
Basic Enclosure                                                    34337.157808
Enhanced Enclosure                                                 35606.210876
Basic Enclosure + HPWH + Hi

In [18]:
metric = "pct_saving_energy.total_%"
print("Community summary:")
get_summary(community_results, metric, lower_percentile=0.25, upper_percentile=0.75)

Community summary:


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,1217,39762.000000,60.973876,53.006521,71.744138
Basic Enclosure + High Efficiency Whole Home Electrification,1217,39762.000000,60.176907,52.214968,71.186900
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,1217,39762.000000,60.095768,51.675971,71.065719
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,1217,39762.000000,59.298833,50.836656,70.780478
High Efficiency Whole Home Electrification,1217,39762.000000,54.329170,44.833602,67.887588
High Efficiency Heat Pump with Electric Heat Backup,1217,39762.000000,43.577378,32.230000,57.450301
Mininum Efficiency Whole Home Electrification,1037,35144.658339,36.754910,27.539903,54.803963
Mininum Efficiency Heat Pump with Electric Heat Backup,1217,39762.000000,28.879330,18.472464,43.076464
Enhanced Enclosure,1088,35606.210876,19.135668,11.316356,26.596441


# 2. Results for Segments (All Income)

Segment Definition

Segment 1. Occupied, Single family detached, pre 1940, renter, all income  

Segment 2. Occupied, Single family detached, pre 1940, owner, all income  

Segment 3. Occupied, Multifamily with 2-4 units, 1940-1979, all income  

Segment 4. Occupied,  Multifamily with 5+ units, 1940-1979, all income 

Segment 5. Occupied,  all building types, 1980+, all income


In [19]:
income = 'all_incomes'
vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Detached']
seg1_vintage = ['<1940']
seg1_tenure = ['Renter']
seg1_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg2_building_type = ['Single-Family Detached']
seg2_vintage = ['<1940']
seg2_tenure = ['Owner']
seg2_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg3_building_type = ['Multi-Family with 2 - 4 Units']
seg3_vintage = ['1940s', '1950s', '1960s', '1970s']
seg3_tenure = ['Renter','Owner']
seg3_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg4_building_type = ['Multi-Family with 5+ Units']
seg4_vintage = ['1940s', '1950s', '1960s', '1970s']
seg4_tenure = ['Renter','Owner']
seg4_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg5_vintage = ['1980s', '1990s', '2000s', '2010s']
seg5_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg1_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg1_tenure) & 
                     community_results['build_existing_model.area_median_income'].isin(seg1_median_income),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg2_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg2_tenure) & 
                     community_results['build_existing_model.area_median_income'].isin(seg2_median_income),
                     'segment'] = 'segment2'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg3_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg3_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg3_tenure) & 
                     community_results['build_existing_model.area_median_income'].isin(seg3_median_income),
                     'segment'] = 'segment3'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg4_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg4_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg4_tenure) & 
                     community_results['build_existing_model.area_median_income'].isin(seg4_median_income),
                     'segment'] = 'segment4'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.vintage'].isin(seg5_vintage) &
                     community_results['build_existing_model.area_median_income'].isin(seg5_median_income),
                     'segment'] = 'segment5'

In [20]:
metric = 'pct_saving_energy.total_%'  # TODO 'pct_saving_cbill.total_%' #'pct_saving_energy.total_%' #'saving_energy.total_mmbtu' #'saving_cbill.total_usd'

for segment in sorted(community_results["segment"].dropna().unique()):
    print(f"Summary for segment: {segment}: ")
    df_seg = community_results.loc[community_results["segment"]==segment]
    display(get_summary(df_seg, metric, lower_percentile=0.25, upper_percentile=0.75))

Summary for segment: segment1: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,15,493.001043,65.246053,54.380030,78.486849
Basic Enclosure + High Efficiency Whole Home Electrification,15,493.001043,65.130862,54.380030,78.486849
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,15,493.001043,64.286411,53.354052,77.839138
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,15,493.001043,64.171220,53.354052,77.742090
High Efficiency Whole Home Electrification,15,493.001043,55.165217,40.954340,74.623988
High Efficiency Heat Pump with Electric Heat Backup,15,493.001043,45.767706,32.375854,64.960676
Mininum Efficiency Whole Home Electrification,15,493.001043,39.708669,29.245815,61.724290
Mininum Efficiency Heat Pump with Electric Heat Backup,15,493.001043,33.349826,24.082183,53.763848
Enhanced Enclosure,15,493.001043,29.184529,23.336893,33.785020


Summary for segment: segment2: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,93,2552.842905,66.769462,59.452152,78.723786
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,93,2552.842905,65.929965,58.065202,77.653925
Basic Enclosure + High Efficiency Whole Home Electrification,93,2552.842905,65.581216,56.870969,78.723786
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,93,2552.842905,64.741797,56.512904,77.653925
High Efficiency Whole Home Electrification,93,2552.842905,57.896240,44.023295,75.687947
High Efficiency Heat Pump with Electric Heat Backup,93,2552.842905,50.105381,33.265348,68.534187
Mininum Efficiency Whole Home Electrification,82,2397.708951,42.872180,30.930000,62.634573
Mininum Efficiency Heat Pump with Electric Heat Backup,93,2552.842905,36.277012,21.474551,55.840627
Enhanced Enclosure,92,2539.746067,25.384161,18.730972,33.315135


Summary for segment: segment3: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,16,1228.155978,59.496866,48.505000,70.916714
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,16,1228.155978,58.217000,48.112656,70.007821
Basic Enclosure + High Efficiency Whole Home Electrification,16,1228.155978,56.723058,47.386225,70.916714
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,16,1228.155978,55.443779,46.676961,70.007821
High Efficiency Whole Home Electrification,16,1228.155978,48.963942,32.658352,69.725036
High Efficiency Heat Pump with Electric Heat Backup,16,1228.155978,38.118031,22.627344,54.242036
Mininum Efficiency Whole Home Electrification,10,853.586413,37.843936,12.199063,58.206810
Mininum Efficiency Heat Pump with Electric Heat Backup,16,1228.155978,23.420356,4.262656,42.870679
Mininum Efficiency Heat Pump with Existing Heat Backup,16,1228.155978,19.721508,2.097578,42.763214


Summary for segment: segment4: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,53,2185.367208,55.552650,47.928259,67.677372
Basic Enclosure + High Efficiency Whole Home Electrification,53,2185.367208,54.496949,47.928259,66.664539
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,53,2185.367208,54.373469,46.481381,66.515319
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,53,2185.367208,53.317878,46.481381,64.464663
High Efficiency Whole Home Electrification,53,2185.367208,50.257786,39.431215,62.957417
High Efficiency Heat Pump with Electric Heat Backup,53,2185.367208,35.532866,23.946493,49.151633
Mininum Efficiency Whole Home Electrification,29,1435.596212,28.237764,6.498430,44.639860
Mininum Efficiency Heat Pump with Existing Heat Backup,52,1969.269382,23.236009,6.921691,38.377883
Mininum Efficiency Heat Pump with Electric Heat Backup,53,2185.367208,22.561790,7.240480,38.515764


Summary for segment: segment5: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,225,7287.889107,53.392880,48.836195,63.457460
Basic Enclosure + High Efficiency Whole Home Electrification,225,7287.889107,52.467162,48.451007,62.580971
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,225,7287.889107,52.073577,48.272469,62.276614
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,225,7287.889107,51.147798,47.516748,61.334831
High Efficiency Whole Home Electrification,225,7287.889107,48.436314,44.482898,59.018343
High Efficiency Heat Pump with Electric Heat Backup,225,7287.889107,33.523501,26.233369,47.792840
Mininum Efficiency Whole Home Electrification,173,5731.426790,27.980566,22.821331,40.457814
Mininum Efficiency Heat Pump with Electric Heat Backup,225,7287.889107,19.103738,12.408210,33.064715
Enhanced Enclosure,172,5590.332477,12.418743,5.590040,18.113259


## 2.1 Upgrade Cost Per Dwelling Unit 

In [21]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]
saving_results[f'segment3_{income}_upgrade_cost_mean_usd'] = results[2]
saving_results[f'segment4_{income}_upgrade_cost_mean_usd'] = results[3]
saving_results[f'segment5_{income}_upgrade_cost_mean_usd'] = results[4]

segment1: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1127.3
Electric Clothes Dryer                                              1266.0
Induction Cooking                                                   1953.1
Heat Pump Clothes Dryer                                             2547.5
Heat Pump Water Heater                                              2977.1
Basic Enclosure                                                     4161.7
Enhanced Enclosure                                                  6039.4
High Efficiency Heat Pump with Electric Heat Backup                34616.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     35306.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        35449.8
Enhanced Enclosure + High Efficiency Whole Home Electrification    39806.8
Basic Enclosure + High Efficiency Whole Home Electrification       39950.3
High Efficiency Whole Home Electrification                         42094.6
Mininum Effi

segment2: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                     1127.3
Electric Clothes Dryer                                               1266.0
Induction Cooking                                                    1953.1
Heat Pump Clothes Dryer                                              2547.5
Heat Pump Water Heater                                               3243.2
Basic Enclosure                                                      4499.4
Enhanced Enclosure                                                   7233.8
High Efficiency Heat Pump with Electric Heat Backup                 43875.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         44714.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      44966.1
Basic Enclosure + High Efficiency Whole Home Electrification        49177.8
Enhanced Enclosure + High Efficiency Whole Home Electrification     49429.2
High Efficiency Whole Home Electrification                          51581.7

segment3: upgrade cost (usd) per dwelling unit


upgrade_name
Basic Enclosure                                                      828.2
Electric Cooking                                                    1127.3
Induction Cooking                                                   1953.1
Enhanced Enclosure                                                  2458.7
Heat Pump Clothes Dryer                                             2547.5
Heat Pump Water Heater                                              2905.1
High Efficiency Heat Pump with Electric Heat Backup                23617.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        26149.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     27683.6
Basic Enclosure + High Efficiency Whole Home Electrification       30000.7
High Efficiency Whole Home Electrification                         30374.7
Enhanced Enclosure + High Efficiency Whole Home Electrification    31535.3
Mininum Efficiency Heat Pump with Electric Heat Backup             36178.5
Mininum Effi

segment4: upgrade cost (usd) per dwelling unit


upgrade_name
Basic Enclosure                                                      708.9
Electric Cooking                                                    1127.3
Induction Cooking                                                   1953.1
Enhanced Enclosure                                                  2244.1
Heat Pump Clothes Dryer                                             2547.5
Heat Pump Water Heater                                              2905.1
High Efficiency Heat Pump with Electric Heat Backup                20402.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        22129.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     22970.4
Basic Enclosure + High Efficiency Whole Home Electrification       24572.6
Mininum Efficiency Whole Home Electrification                      25374.0
Enhanced Enclosure + High Efficiency Whole Home Electrification    25413.4
High Efficiency Whole Home Electrification                         25750.1
Mininum Effi

segment5: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1127.3
Electric Clothes Dryer                                              1266.0
Induction Cooking                                                   1953.1
Basic Enclosure                                                     2103.1
Heat Pump Clothes Dryer                                             2547.5
Enhanced Enclosure                                                  2735.3
Heat Pump Water Heater                                              3100.7
High Efficiency Heat Pump with Electric Heat Backup                24287.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        27898.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     28144.9
High Efficiency Whole Home Electrification                         31013.3
Mininum Efficiency Whole Home Electrification                      31143.5
Basic Enclosure + High Efficiency Whole Home Electrification       31523.5
Enhanced Enc

 ## 2.2 Annual Savings Per Dollar Invested

 ### Annual Bill Savings Per Dollar Invested

In [22]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_per_dollar_invested_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_per_dollar_invested_usd'] = results[3]
saving_results[f'segment5_{income}_saving_bill_per_dollar_invested_usd'] = results[4]

segment1


upgrade_name
Basic Enclosure                                                    0.08664
Enhanced Enclosure                                                 0.07478
Heat Pump Water Heater                                             0.03586
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.01085
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.01067
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01027
Basic Enclosure + High Efficiency Whole Home Electrification       0.01012
Heat Pump Clothes Dryer                                            0.00791
Induction Cooking                                                  0.00126
Mininum Efficiency Heat Pump with Existing Heat Backup            -0.00000
High Efficiency Whole Home Electrification                        -0.00159
Electric Cooking                                                  -0.00525
High Efficiency Heat Pump with Electric Heat Backup               -0.00730
Mininum Effi

segment2


upgrade_name
Basic Enclosure                                                    0.09081
Enhanced Enclosure                                                 0.07420
Heat Pump Water Heater                                             0.01918
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.01430
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01349
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.01264
Basic Enclosure + High Efficiency Whole Home Electrification       0.01197
Heat Pump Clothes Dryer                                            0.00777
High Efficiency Whole Home Electrification                         0.00270
Induction Cooking                                                  0.00045
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00031
High Efficiency Heat Pump with Electric Heat Backup               -0.00039
Mininum Efficiency Heat Pump with Electric Heat Backup            -0.00527
Electric Coo

segment3


upgrade_name
Basic Enclosure                                                    0.23262
Enhanced Enclosure                                                 0.09885
Heat Pump Water Heater                                             0.02324
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02266
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02147
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02045
Basic Enclosure + High Efficiency Whole Home Electrification       0.01930
High Efficiency Whole Home Electrification                         0.01279
High Efficiency Heat Pump with Electric Heat Backup                0.01031
Heat Pump Clothes Dryer                                            0.00722
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00226
Induction Cooking                                                  0.00058
Mininum Efficiency Heat Pump with Electric Heat Backup            -0.00240
Mininum Effi

segment4


upgrade_name
Basic Enclosure                                                    0.23895
Enhanced Enclosure                                                 0.08876
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02140
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02136
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01959
Basic Enclosure + High Efficiency Whole Home Electrification       0.01957
High Efficiency Whole Home Electrification                         0.01596
High Efficiency Heat Pump with Electric Heat Backup                0.01557
Heat Pump Water Heater                                             0.01523
Heat Pump Clothes Dryer                                            0.00728
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00375
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00278
Induction Cooking                                                  0.00028
Mininum Effi

segment5


upgrade_name
Basic Enclosure                                                    0.07532
Enhanced Enclosure                                                 0.06513
Heat Pump Water Heater                                             0.01310
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.01176
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.01092
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01089
Basic Enclosure + High Efficiency Whole Home Electrification       0.01015
Heat Pump Clothes Dryer                                            0.00735
High Efficiency Whole Home Electrification                         0.00656
High Efficiency Heat Pump with Electric Heat Backup                0.00343
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00108
Induction Cooking                                                  0.00013
Mininum Efficiency Heat Pump with Electric Heat Backup            -0.00618
Electric Coo

 ### Annual Energy Savings Per Dollar Invested

In [23]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[3]
saving_results[f'segment5_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[4]

segment1


upgrade_name
Basic Enclosure                                                    0.01210
Enhanced Enclosure                                                 0.00975
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00363
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00361
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00326
Basic Enclosure + High Efficiency Whole Home Electrification       0.00325
High Efficiency Heat Pump with Electric Heat Backup                0.00270
High Efficiency Whole Home Electrification                         0.00263
Heat Pump Water Heater                                             0.00248
Electric Cooking                                                   0.00135
Mininum Efficiency Whole Home Electrification                      0.00115
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00093
Electric Clothes Dryer                                             0.00055
Heat Pump Cl

segment2


upgrade_name
Basic Enclosure                                                    0.01038
Enhanced Enclosure                                                 0.00826
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00341
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00335
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00313
Basic Enclosure + High Efficiency Whole Home Electrification       0.00308
High Efficiency Heat Pump with Electric Heat Backup                0.00267
High Efficiency Whole Home Electrification                         0.00258
Heat Pump Water Heater                                             0.00242
Electric Cooking                                                   0.00131
Mininum Efficiency Whole Home Electrification                      0.00075
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00062
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00048
Heat Pump Cl

segment3


upgrade_name
Basic Enclosure                                                    0.01539
Enhanced Enclosure                                                 0.00666
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00213
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00211
Basic Enclosure + High Efficiency Whole Home Electrification       0.00189
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00189
Heat Pump Water Heater                                             0.00171
High Efficiency Heat Pump with Electric Heat Backup                0.00163
High Efficiency Whole Home Electrification                         0.00161
Electric Cooking                                                   0.00121
Mininum Efficiency Whole Home Electrification                      0.00084
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00069
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00054
Heat Pump Cl

segment4


upgrade_name
Basic Enclosure                                                    0.01331
Enhanced Enclosure                                                 0.00514
Heat Pump Water Heater                                             0.00147
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00145
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00143
Basic Enclosure + High Efficiency Whole Home Electrification       0.00133
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00131
Electric Cooking                                                   0.00117
High Efficiency Whole Home Electrification                         0.00116
High Efficiency Heat Pump with Electric Heat Backup                0.00109
Mininum Efficiency Whole Home Electrification                      0.00076
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00030
Heat Pump Clothes Dryer                                            0.00030
Mininum Effi

segment5


upgrade_name
Basic Enclosure                                                    0.00685
Enhanced Enclosure                                                 0.00577
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00211
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00209
Heat Pump Water Heater                                             0.00192
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00191
Basic Enclosure + High Efficiency Whole Home Electrification       0.00189
High Efficiency Whole Home Electrification                         0.00177
High Efficiency Heat Pump with Electric Heat Backup                0.00173
Electric Cooking                                                   0.00140
Mininum Efficiency Whole Home Electrification                      0.00125
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00055
Electric Clothes Dryer                                             0.00047
Heat Pump Cl

 ### Annual Emission Savings Per Dollar Invested

In [24]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[3]
saving_results[f'segment5_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[4]

segment1


upgrade_name
Basic Enclosure                                                    0.87198
Enhanced Enclosure                                                 0.69191
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.28919
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.28782
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.25900
Basic Enclosure + High Efficiency Whole Home Electrification       0.25789
High Efficiency Heat Pump with Electric Heat Backup                0.24475
High Efficiency Whole Home Electrification                         0.22747
Heat Pump Water Heater                                             0.13478
Electric Cooking                                                   0.12935
Mininum Efficiency Whole Home Electrification                      0.12108
Mininum Efficiency Heat Pump with Electric Heat Backup             0.10193
Electric Clothes Dryer                                             0.09952
Mininum Effi

segment2


upgrade_name
Basic Enclosure                                                    0.70827
Enhanced Enclosure                                                 0.55923
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.26055
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.25929
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.23902
Basic Enclosure + High Efficiency Whole Home Electrification       0.23776
High Efficiency Heat Pump with Electric Heat Backup                0.22580
High Efficiency Whole Home Electrification                         0.21305
Heat Pump Water Heater                                             0.16571
Electric Cooking                                                   0.12633
Mininum Efficiency Whole Home Electrification                      0.07453
Electric Clothes Dryer                                             0.06480
Mininum Efficiency Heat Pump with Electric Heat Backup             0.06260
Mininum Effi

segment3


upgrade_name
Basic Enclosure                                                    0.86086
Enhanced Enclosure                                                 0.37614
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.13871
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.13481
Basic Enclosure + High Efficiency Whole Home Electrification       0.12310
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.12044
High Efficiency Heat Pump with Electric Heat Backup                0.11751
Electric Cooking                                                   0.11674
High Efficiency Whole Home Electrification                         0.11165
Heat Pump Water Heater                                             0.09882
Mininum Efficiency Whole Home Electrification                      0.07668
Mininum Efficiency Heat Pump with Electric Heat Backup             0.06279
Mininum Efficiency Heat Pump with Existing Heat Backup             0.04161
Induction Co

segment4


upgrade_name
Basic Enclosure                                                    0.66894
Enhanced Enclosure                                                 0.26478
Electric Cooking                                                   0.12003
Heat Pump Water Heater                                             0.09554
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.08159
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.07975
Basic Enclosure + High Efficiency Whole Home Electrification       0.07535
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.07389
High Efficiency Whole Home Electrification                         0.06769
Mininum Efficiency Whole Home Electrification                      0.06642
High Efficiency Heat Pump with Electric Heat Backup                0.06233
Induction Cooking                                                  0.02228
Mininum Efficiency Heat Pump with Electric Heat Backup             0.01997
Mininum Effi

segment5


upgrade_name
Basic Enclosure                                                    0.43410
Enhanced Enclosure                                                 0.36269
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.15458
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.15447
Electric Cooking                                                   0.14014
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.13960
Basic Enclosure + High Efficiency Whole Home Electrification       0.13939
High Efficiency Heat Pump with Electric Heat Backup                0.13799
Heat Pump Water Heater                                             0.13798
High Efficiency Whole Home Electrification                         0.13582
Mininum Efficiency Whole Home Electrification                      0.13541
Electric Clothes Dryer                                             0.08571
Mininum Efficiency Heat Pump with Electric Heat Backup             0.05728
Induction Co

## 2.3 Annual Total Savings

### Annual Total Bill Savings

In [25]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_total_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_total_usd'] = results[3]
saving_results[f'segment5_{income}_saving_bill_total_usd'] = results[4]

segment1


upgrade_name
Enhanced Enclosure                                                 222662.8
Enhanced Enclosure + High Efficiency Whole Home Electrification    201613.5
Basic Enclosure + High Efficiency Whole Home Electrification       199309.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     188788.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        186484.7
Basic Enclosure                                                    177753.5
Heat Pump Water Heater                                              52634.0
Heat Pump Clothes Dryer                                              9938.1
Induction Cooking                                                    1215.6
Mininum Efficiency Heat Pump with Existing Heat Backup               -151.9
Electric Cooking                                                     -554.4
Electric Clothes Dryer                                              -3466.9
High Efficiency Whole Home Electrification                         -32902.0

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1702746.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1641928.1
Basic Enclosure + High Efficiency Whole Home Electrification       1503287.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1442468.7
Enhanced Enclosure                                                 1363273.7
Basic Enclosure                                                    1022104.2
High Efficiency Whole Home Electrification                          355325.4
Heat Pump Water Heater                                              158827.4
Mininum Efficiency Heat Pump with Existing Heat Backup               74970.4
Heat Pump Clothes Dryer                                              49791.5
Induction Cooking                                                     2231.8
Electric Cooking                                                     -6084.8
Electric Clothes Dryer                                         

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    792211.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     770489.3
Basic Enclosure + High Efficiency Whole Home Electrification       711236.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        689515.0
High Efficiency Whole Home Electrification                         477308.0
High Efficiency Heat Pump with Electric Heat Backup                299108.1
Enhanced Enclosure                                                 298505.2
Basic Enclosure                                                    236605.8
Mininum Efficiency Heat Pump with Existing Heat Backup             100338.1
Heat Pump Water Heater                                              82933.8
Heat Pump Clothes Dryer                                             16834.4
Induction Cooking                                                    1379.8
Electric Cooking                                                    -2188.9

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1087922.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1072307.1
Basic Enclosure + High Efficiency Whole Home Electrification       1050678.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1035063.1
High Efficiency Whole Home Electrification                          898240.3
High Efficiency Heat Pump with Electric Heat Backup                 694263.8
Mininum Efficiency Heat Pump with Existing Heat Backup              398909.1
Enhanced Enclosure                                                  342380.1
Mininum Efficiency Heat Pump with Electric Heat Backup              302858.5
Basic Enclosure                                                     275923.1
Heat Pump Water Heater                                               96669.9
Heat Pump Clothes Dryer                                              10859.0
Induction Cooking                                              

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    2521154.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     2411390.1
Basic Enclosure + High Efficiency Whole Home Electrification       2330772.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        2221008.6
High Efficiency Whole Home Electrification                         1483494.5
Enhanced Enclosure                                                  995944.7
Basic Enclosure                                                     840917.2
High Efficiency Heat Pump with Electric Heat Backup                 606754.3
Mininum Efficiency Heat Pump with Existing Heat Backup              343653.0
Heat Pump Water Heater                                              296098.7
Heat Pump Clothes Dryer                                              89546.4
Induction Cooking                                                     1886.2
Electric Cooking                                               

### Annual Total Energy Savings

In [26]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_total_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_total_mmbtu'] = results[3]
saving_results[f'segment5_{income}_saving_energy_total_mmbtu'] = results[4]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    64054.9
Basic Enclosure + High Efficiency Whole Home Electrification       63961.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     63209.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        63115.9
High Efficiency Whole Home Electrification                         54510.5
High Efficiency Heat Pump with Electric Heat Backup                46064.6
Mininum Efficiency Whole Home Electrification                      39743.8
Mininum Efficiency Heat Pump with Electric Heat Backup             33949.9
Enhanced Enclosure                                                 29033.9
Basic Enclosure                                                    24820.3
Mininum Efficiency Heat Pump with Existing Heat Backup             16412.7
Heat Pump Water Heater                                              3635.8
Heat Pump Clothes Dryer                                              630.4
Induction Co

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    395101.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     390885.5
Basic Enclosure + High Efficiency Whole Home Electrification       386983.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        382767.8
High Efficiency Whole Home Electrification                         340263.9
High Efficiency Heat Pump with Electric Heat Backup                298750.3
Mininum Efficiency Whole Home Electrification                      247686.7
Mininum Efficiency Heat Pump with Electric Heat Backup             223519.7
Enhanced Enclosure                                                 151681.1
Mininum Efficiency Heat Pump with Existing Heat Backup             117264.3
Basic Enclosure                                                    116797.1
Heat Pump Water Heater                                              20022.3
Heat Pump Clothes Dryer                                              2472.0

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    73062.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     71675.9
Basic Enclosure + High Efficiency Whole Home Electrification       69767.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        68380.4
High Efficiency Whole Home Electrification                         60246.6
High Efficiency Heat Pump with Electric Heat Backup                47349.0
Mininum Efficiency Whole Home Electrification                      35928.8
Mininum Efficiency Heat Pump with Electric Heat Backup             30830.6
Mininum Efficiency Heat Pump with Existing Heat Backup             24037.3
Enhanced Enclosure                                                 20102.7
Basic Enclosure                                                    15648.4
Heat Pump Water Heater                                              6094.6
Induction Cooking                                                    701.6
Heat Pump Cl

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    73017.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     71563.6
Basic Enclosure + High Efficiency Whole Home Electrification       71501.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        70047.8
High Efficiency Whole Home Electrification                         65297.4
High Efficiency Heat Pump with Electric Heat Backup                48540.7
Mininum Efficiency Heat Pump with Electric Heat Backup             32614.5
Mininum Efficiency Heat Pump with Existing Heat Backup             28262.0
Mininum Efficiency Whole Home Electrification                      27755.5
Enhanced Enclosure                                                 19814.1
Basic Enclosure                                                    15374.3
Heat Pump Water Heater                                              9355.3
Induction Cooking                                                   1011.6
Electric Coo

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    441913.0
Basic Enclosure + High Efficiency Whole Home Electrification       434164.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     432478.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        424730.0
High Efficiency Whole Home Electrification                         399682.3
High Efficiency Heat Pump with Electric Heat Backup                306167.1
Mininum Efficiency Whole Home Electrification                      222825.9
Mininum Efficiency Heat Pump with Electric Heat Backup             193023.0
Enhanced Enclosure                                                  88234.8
Basic Enclosure                                                     76459.5
Mininum Efficiency Heat Pump with Existing Heat Backup              52017.7
Heat Pump Water Heater                                              43429.2
Heat Pump Clothes Dryer                                              5076.4

### Annual Total Emission Savings

In [27]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_total_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_total_kgCO2e'] = results[3]
saving_results[f'segment5_{income}_saving_emission_total_kgCO2e'] = results[4]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    5082917.7
Basic Enclosure + High Efficiency Whole Home Electrification       5079360.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     5033704.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        5030146.5
High Efficiency Whole Home Electrification                         4720578.1
Mininum Efficiency Whole Home Electrification                      4185951.1
High Efficiency Heat Pump with Electric Heat Backup                4177014.3
Mininum Efficiency Heat Pump with Electric Heat Backup             3731688.3
Enhanced Enclosure                                                 2060102.9
Basic Enclosure                                                    1789076.9
Mininum Efficiency Heat Pump with Existing Heat Backup             1399874.0
Heat Pump Water Heater                                              197824.3
Heat Pump Clothes Dryer                                        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    30160900.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     29909306.1
Basic Enclosure + High Efficiency Whole Home Electrification       29849149.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        29597555.6
High Efficiency Whole Home Electrification                         28054961.4
High Efficiency Heat Pump with Electric Heat Backup                25290857.8
Mininum Efficiency Whole Home Electrification                      24656099.8
Mininum Efficiency Heat Pump with Electric Heat Backup             22491493.0
Enhanced Enclosure                                                 10274190.1
Mininum Efficiency Heat Pump with Existing Heat Backup              9832013.3
Basic Enclosure                                                     7972131.0
Heat Pump Water Heater                                              1371995.5
Induction Cooking                                  

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4664583.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4583676.3
Basic Enclosure + High Efficiency Whole Home Electrification       4535551.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4454644.4
High Efficiency Whole Home Electrification                         4165074.2
High Efficiency Heat Pump with Electric Heat Backup                3408567.7
Mininum Efficiency Whole Home Electrification                      3276294.6
Mininum Efficiency Heat Pump with Electric Heat Backup             2789868.9
Mininum Efficiency Heat Pump with Existing Heat Backup             1848740.6
Enhanced Enclosure                                                 1135807.3
Basic Enclosure                                                     875600.1
Heat Pump Water Heater                                              352573.1
Induction Cooking                                              

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4103929.1
Basic Enclosure + High Efficiency Whole Home Electrification       4046354.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4003587.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3946013.1
High Efficiency Whole Home Electrification                         3808863.6
High Efficiency Heat Pump with Electric Heat Backup                2779096.9
Mininum Efficiency Whole Home Electrification                      2419544.8
Mininum Efficiency Heat Pump with Electric Heat Backup             2178019.2
Mininum Efficiency Heat Pump with Existing Heat Backup             1630701.7
Enhanced Enclosure                                                 1021333.8
Basic Enclosure                                                     772435.0
Heat Pump Water Heater                                              606537.1
Induction Cooking                                              

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    32322934.3
Basic Enclosure + High Efficiency Whole Home Electrification       32023312.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     31707087.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        31407464.9
High Efficiency Whole Home Electrification                         30698268.5
High Efficiency Heat Pump with Electric Heat Backup                24424359.5
Mininum Efficiency Whole Home Electrification                      24171104.8
Mininum Efficiency Heat Pump with Electric Heat Backup             20156850.2
Enhanced Enclosure                                                  5545905.3
Basic Enclosure                                                     4846673.7
Mininum Efficiency Heat Pump with Existing Heat Backup              3794715.6
Heat Pump Water Heater                                              3118070.9
Induction Cooking                                  

## 2.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [28]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_mean_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_mean_usd'] = results[3]
saving_results[f'segment5_{income}_saving_bill_mean_usd'] = results[4]

segment1


upgrade_name
Enhanced Enclosure                                                 451.6
Enhanced Enclosure + High Efficiency Whole Home Electrification    409.0
Basic Enclosure + High Efficiency Whole Home Electrification       404.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     382.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        378.3
Basic Enclosure                                                    360.6
Heat Pump Water Heater                                             106.8
Heat Pump Clothes Dryer                                             20.2
Induction Cooking                                                    2.5
Mininum Efficiency Heat Pump with Existing Heat Backup              -0.3
Electric Cooking                                                    -5.9
Electric Clothes Dryer                                             -28.2
High Efficiency Whole Home Electrification                         -66.7
High Efficiency Heat Pump with Electri

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    667.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     643.2
Basic Enclosure + High Efficiency Whole Home Electrification       588.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        565.0
Enhanced Enclosure                                                 536.8
Basic Enclosure                                                    408.6
High Efficiency Whole Home Electrification                         139.2
Heat Pump Water Heater                                              62.2
Mininum Efficiency Heat Pump with Existing Heat Backup              31.7
Heat Pump Clothes Dryer                                             19.8
Induction Cooking                                                    0.9
Electric Cooking                                                    -6.4
High Efficiency Heat Pump with Electric Heat Backup                -17.0
Electric Clothes Dryer                

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    645.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     627.4
Basic Enclosure + High Efficiency Whole Home Electrification       579.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        561.4
High Efficiency Whole Home Electrification                         388.6
High Efficiency Heat Pump with Electric Heat Backup                243.5
Enhanced Enclosure                                                 243.1
Basic Enclosure                                                    192.7
Mininum Efficiency Heat Pump with Existing Heat Backup              81.7
Heat Pump Water Heater                                              67.5
Heat Pump Clothes Dryer                                             18.4
Induction Cooking                                                    1.1
Electric Cooking                                                    -5.4
Mininum Efficiency Heat Pump with Elec

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    497.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     490.7
Basic Enclosure + High Efficiency Whole Home Electrification       480.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        473.6
High Efficiency Whole Home Electrification                         411.0
High Efficiency Heat Pump with Electric Heat Backup                317.7
Mininum Efficiency Heat Pump with Existing Heat Backup             202.6
Enhanced Enclosure                                                 199.2
Basic Enclosure                                                    169.4
Mininum Efficiency Heat Pump with Electric Heat Backup             138.6
Heat Pump Water Heater                                              44.2
Heat Pump Clothes Dryer                                             18.5
Induction Cooking                                                    0.5
Electric Cooking                      

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    345.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     330.9
Basic Enclosure + High Efficiency Whole Home Electrification       319.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        304.8
High Efficiency Whole Home Electrification                         203.6
Enhanced Enclosure                                                 178.2
Basic Enclosure                                                    158.4
High Efficiency Heat Pump with Electric Heat Backup                 83.3
Mininum Efficiency Heat Pump with Existing Heat Backup              48.6
Heat Pump Water Heater                                              40.6
Heat Pump Clothes Dryer                                             18.7
Induction Cooking                                                    0.3
Electric Cooking                                                   -10.0
Electric Clothes Dryer                

### Annual Energy Savings Per Dwelling Unit

In [29]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_mean_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_mean_mmbtu'] = results[3]
saving_results[f'segment5_{income}_saving_energy_mean_mmbtu'] = results[4]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    129.9
Basic Enclosure + High Efficiency Whole Home Electrification       129.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     128.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        128.0
High Efficiency Whole Home Electrification                         110.6
High Efficiency Heat Pump with Electric Heat Backup                 93.4
Mininum Efficiency Whole Home Electrification                       80.6
Mininum Efficiency Heat Pump with Electric Heat Backup              68.9
Enhanced Enclosure                                                  58.9
Basic Enclosure                                                     50.3
Mininum Efficiency Heat Pump with Existing Heat Backup              34.1
Heat Pump Water Heater                                               7.4
Electric Cooking                                                     1.5
Heat Pump Clothes Dryer               

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    154.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     153.1
Basic Enclosure + High Efficiency Whole Home Electrification       151.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        149.9
High Efficiency Whole Home Electrification                         133.3
High Efficiency Heat Pump with Electric Heat Backup                117.0
Mininum Efficiency Whole Home Electrification                      103.3
Mininum Efficiency Heat Pump with Electric Heat Backup              87.6
Enhanced Enclosure                                                  59.7
Mininum Efficiency Heat Pump with Existing Heat Backup              49.6
Basic Enclosure                                                     46.7
Heat Pump Water Heater                                               7.8
Electric Cooking                                                     1.5
Heat Pump Clothes Dryer               

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    59.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     58.4
Basic Enclosure + High Efficiency Whole Home Electrification       56.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        55.7
High Efficiency Whole Home Electrification                         49.1
Mininum Efficiency Whole Home Electrification                      42.1
High Efficiency Heat Pump with Electric Heat Backup                38.6
Mininum Efficiency Heat Pump with Electric Heat Backup             25.1
Mininum Efficiency Heat Pump with Existing Heat Backup             19.6
Enhanced Enclosure                                                 16.4
Basic Enclosure                                                    12.7
Heat Pump Water Heater                                              5.0
Electric Cooking                                                    1.4
Heat Pump Clothes Dryer                            

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    33.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     32.7
Basic Enclosure + High Efficiency Whole Home Electrification       32.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        32.1
High Efficiency Whole Home Electrification                         29.9
High Efficiency Heat Pump with Electric Heat Backup                22.2
Mininum Efficiency Whole Home Electrification                      19.3
Mininum Efficiency Heat Pump with Electric Heat Backup             14.9
Mininum Efficiency Heat Pump with Existing Heat Backup             14.4
Enhanced Enclosure                                                 11.5
Basic Enclosure                                                     9.4
Heat Pump Water Heater                                              4.3
Electric Cooking                                                    1.3
Heat Pump Clothes Dryer                            

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    60.6
Basic Enclosure + High Efficiency Whole Home Electrification       59.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     59.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        58.3
High Efficiency Whole Home Electrification                         54.8
High Efficiency Heat Pump with Electric Heat Backup                42.0
Mininum Efficiency Whole Home Electrification                      38.9
Mininum Efficiency Heat Pump with Electric Heat Backup             26.5
Enhanced Enclosure                                                 15.8
Basic Enclosure                                                    14.4
Mininum Efficiency Heat Pump with Existing Heat Backup              7.4
Heat Pump Water Heater                                              6.0
Electric Cooking                                                    1.6
Heat Pump Clothes Dryer                            

### Annual Percentage Energy Savings Per Dwelling Unit

In [30]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]
saving_results[f'segment3_{income}_pct_saving_energy_mean_%'] = results[2]
saving_results[f'segment4_{income}_pct_saving_energy_mean_%'] = results[3]
saving_results[f'segment5_{income}_pct_saving_energy_mean_%'] = results[4]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    65.2
Basic Enclosure + High Efficiency Whole Home Electrification       65.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     64.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        64.2
High Efficiency Whole Home Electrification                         55.2
High Efficiency Heat Pump with Electric Heat Backup                45.8
Mininum Efficiency Whole Home Electrification                      39.7
Mininum Efficiency Heat Pump with Electric Heat Backup             33.3
Enhanced Enclosure                                                 29.2
Basic Enclosure                                                    24.1
Mininum Efficiency Heat Pump with Existing Heat Backup             15.8
Heat Pump Water Heater                                              4.5
Electric Cooking                                                    0.9
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    66.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     65.9
Basic Enclosure + High Efficiency Whole Home Electrification       65.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        64.7
High Efficiency Whole Home Electrification                         57.9
High Efficiency Heat Pump with Electric Heat Backup                50.1
Mininum Efficiency Whole Home Electrification                      42.9
Mininum Efficiency Heat Pump with Electric Heat Backup             36.3
Enhanced Enclosure                                                 25.4
Mininum Efficiency Heat Pump with Existing Heat Backup             22.3
Basic Enclosure                                                    19.3
Heat Pump Water Heater                                              3.8
Electric Cooking                                                    0.7
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    59.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     58.2
Basic Enclosure + High Efficiency Whole Home Electrification       56.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        55.4
High Efficiency Whole Home Electrification                         49.0
High Efficiency Heat Pump with Electric Heat Backup                38.1
Mininum Efficiency Whole Home Electrification                      37.8
Mininum Efficiency Heat Pump with Electric Heat Backup             23.4
Mininum Efficiency Heat Pump with Existing Heat Backup             19.7
Enhanced Enclosure                                                 15.3
Basic Enclosure                                                    11.5
Heat Pump Water Heater                                              5.3
Electric Cooking                                                    1.5
Heat Pump Clothes Dryer                            

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    55.6
Basic Enclosure + High Efficiency Whole Home Electrification       54.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     54.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        53.3
High Efficiency Whole Home Electrification                         50.3
High Efficiency Heat Pump with Electric Heat Backup                35.5
Mininum Efficiency Whole Home Electrification                      28.2
Mininum Efficiency Heat Pump with Existing Heat Backup             23.2
Mininum Efficiency Heat Pump with Electric Heat Backup             22.6
Enhanced Enclosure                                                 17.6
Basic Enclosure                                                    14.1
Heat Pump Water Heater                                              8.8
Electric Cooking                                                    2.3
Heat Pump Clothes Dryer                            

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    53.4
Basic Enclosure + High Efficiency Whole Home Electrification       52.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     52.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        51.1
High Efficiency Whole Home Electrification                         48.4
High Efficiency Heat Pump with Electric Heat Backup                33.5
Mininum Efficiency Whole Home Electrification                      28.0
Mininum Efficiency Heat Pump with Electric Heat Backup             19.1
Enhanced Enclosure                                                 12.4
Basic Enclosure                                                    10.9
Mininum Efficiency Heat Pump with Existing Heat Backup              9.4
Heat Pump Water Heater                                              7.3
Electric Cooking                                                    1.6
Heat Pump Clothes Dryer                            

### Annual Emission Savings Per Dwelling Unit

In [31]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_mean_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_mean_kgCO2e'] = results[3]
saving_results[f'segment5_{income}_saving_emission_mean_kgCO2e'] = results[4]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    10310.2
Basic Enclosure + High Efficiency Whole Home Electrification       10302.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     10210.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        10203.1
High Efficiency Whole Home Electrification                          9575.2
Mininum Efficiency Whole Home Electrification                       8490.8
High Efficiency Heat Pump with Electric Heat Backup                 8472.6
Mininum Efficiency Heat Pump with Electric Heat Backup              7569.3
Enhanced Enclosure                                                  4178.7
Basic Enclosure                                                     3629.0
Mininum Efficiency Heat Pump with Existing Heat Backup              2910.4
Heat Pump Water Heater                                               401.3
Electric Cooking                                                     145.8
Electric Clo

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    11814.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     11716.1
Basic Enclosure + High Efficiency Whole Home Electrification       11692.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        11594.0
High Efficiency Whole Home Electrification                         10989.7
Mininum Efficiency Whole Home Electrification                      10283.2
High Efficiency Heat Pump with Electric Heat Backup                 9906.9
Mininum Efficiency Heat Pump with Electric Heat Backup              8810.4
Mininum Efficiency Heat Pump with Existing Heat Backup              4156.7
Enhanced Enclosure                                                  4045.4
Basic Enclosure                                                     3186.8
Heat Pump Water Heater                                               537.4
Electric Cooking                                                     142.4
Electric Clo

segment3


upgrade_name
Mininum Efficiency Whole Home Electrification                      3838.3
Enhanced Enclosure + High Efficiency Whole Home Electrification    3798.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3732.2
Basic Enclosure + High Efficiency Whole Home Electrification       3693.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3627.1
High Efficiency Whole Home Electrification                         3391.3
High Efficiency Heat Pump with Electric Heat Backup                2775.4
Mininum Efficiency Heat Pump with Electric Heat Backup             2271.6
Mininum Efficiency Heat Pump with Existing Heat Backup             1505.3
Enhanced Enclosure                                                  924.8
Basic Enclosure                                                     712.9
Heat Pump Water Heater                                              287.1
Electric Cooking                                                    131.6
Induction Cooking        

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1877.9
Basic Enclosure + High Efficiency Whole Home Electrification       1851.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1832.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1805.7
High Efficiency Whole Home Electrification                         1742.9
Mininum Efficiency Whole Home Electrification                      1685.4
High Efficiency Heat Pump with Electric Heat Backup                1271.7
Mininum Efficiency Heat Pump with Electric Heat Backup              996.6
Mininum Efficiency Heat Pump with Existing Heat Backup              828.1
Enhanced Enclosure                                                  594.2
Basic Enclosure                                                     474.2
Heat Pump Water Heater                                              277.5
Electric Cooking                                                    135.3
Induction Cooking        

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4435.2
Basic Enclosure + High Efficiency Whole Home Electrification       4394.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4350.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4309.5
Mininum Efficiency Whole Home Electrification                      4217.3
High Efficiency Whole Home Electrification                         4212.2
High Efficiency Heat Pump with Electric Heat Backup                3351.4
Mininum Efficiency Heat Pump with Electric Heat Backup             2765.8
Enhanced Enclosure                                                  992.1
Basic Enclosure                                                     913.0
Mininum Efficiency Heat Pump with Existing Heat Backup              537.2
Heat Pump Water Heater                                              427.8
Electric Cooking                                                    158.0
Electric Clothes Dryer   

## 2.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [32]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 2

In [33]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 3

In [34]:
segment = 'segment3'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment3_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment3_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment3_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 4

In [35]:
segment = 'segment4'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment4_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment4_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment4_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 5

In [36]:
segment = 'segment5'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment5_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment5_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment5_{income}_pct_energy_positive_savings_%'] = results[2]*100

## 2.6. Average Energy Burden (%) Before and after Upgrade

In [37]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]
saving_results[[f'segment3_{income}_energy_burden_before_upgrade_%',f'segment3_{income}_energy_burden_after_upgrade_%']] = results[2]
saving_results[[f'segment4_{income}_energy_burden_before_upgrade_%',f'segment4_{income}_energy_burden_after_upgrade_%']] = results[3]
saving_results[[f'segment5_{income}_energy_burden_before_upgrade_%',f'segment5_{income}_energy_burden_after_upgrade_%']] = results[4]

energy burden (%) before and after upgrade
segment1


/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/2591935474.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/2591935474.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/2591935474.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

,before,after
upgrade_name,,
Enhanced Enclosure,13.1,10.0
Basic Enclosure,13.1,10.3
Enhanced Enclosure + High Efficiency Whole Home Electrification,13.1,12.3
Basic Enclosure + High Efficiency Whole Home Electrification,13.1,12.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,13.1,12.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,13.1,12.5
Heat Pump Water Heater,13.1,12.8
Heat Pump Clothes Dryer,13.1,12.9
Induction Cooking,13.1,13.0


segment2


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,8.2,5.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,8.2,5.5
Basic Enclosure + High Efficiency Whole Home Electrification,8.2,5.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,8.2,5.8
Enhanced Enclosure,8.3,6.7
High Efficiency Whole Home Electrification,8.2,7.0
Basic Enclosure,8.3,7.2
High Efficiency Heat Pump with Electric Heat Backup,8.2,7.5
Mininum Efficiency Heat Pump with Existing Heat Backup,7.8,7.5


...

,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,8.0,4.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,8.0,4.9
Basic Enclosure + High Efficiency Whole Home Electrification,8.0,5.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,8.0,5.5
High Efficiency Whole Home Electrification,8.0,6.2
Enhanced Enclosure,8.0,6.8
High Efficiency Heat Pump with Electric Heat Backup,8.0,7.0
Basic Enclosure,8.0,7.1
Mininum Efficiency Heat Pump with Existing Heat Backup,8.0,7.6


segment4


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,7.9,4.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,7.9,4.1
Basic Enclosure + High Efficiency Whole Home Electrification,7.9,4.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,7.9,4.3
High Efficiency Whole Home Electrification,7.9,4.8
High Efficiency Heat Pump with Electric Heat Backup,7.9,5.5
Enhanced Enclosure,8.9,7.4
Mininum Efficiency Heat Pump with Existing Heat Backup,8.6,7.2
Basic Enclosure,9.2,8.0


segment5


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,6.2,4.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,6.2,4.5
Basic Enclosure + High Efficiency Whole Home Electrification,6.2,4.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,6.2,4.6
High Efficiency Whole Home Electrification,6.2,5.0
Enhanced Enclosure,6.2,5.4
High Efficiency Heat Pump with Electric Heat Backup,6.2,5.5
Basic Enclosure,6.1,5.5
Mininum Efficiency Heat Pump with Existing Heat Backup,6.2,5.9


# 3. Results for Segments (0-80% AMI)

Segment Definition

Segment 1. Occupied, Single family detached, pre 1940, renter, 0-80% AMI  

Segment 2. Occupied, Single family detached, pre 1940, owner, 0-80% AMI 

Segment 3. Occupied, Multifamily with 2-4 units, 1940-1979, 0-80% AMI 

Segment 4. Occupied,  Multifamily with 5+ units, 1940-1979, 0-80% AMI

Segment 5. Occupied,  all building types, 1980+, 0-80% AMI

In [38]:
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")

income = '0-80%AMI'
vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Detached']
seg1_vintage = ['<1940']
seg1_tenure = ['Renter']
seg1_median_income = ['0-30%','30-60%','60-80%']

seg2_building_type = ['Single-Family Detached']
seg2_vintage = ['<1940']
seg2_tenure = ['Owner']
seg2_median_income = ['0-30%','30-60%','60-80%']

seg3_building_type = ['Multi-Family with 2 - 4 Units']
seg3_vintage = ['1940s', '1950s', '1960s', '1970s']
seg3_tenure = ['Renter','Owner']
seg3_median_income = ['0-30%','30-60%','60-80%']

seg4_building_type = ['Multi-Family with 5+ Units']
seg4_vintage = ['1940s', '1950s', '1960s', '1970s']
seg4_tenure = ['Renter','Owner']
seg4_median_income = ['0-30%','30-60%','60-80%']

seg5_vintage = ['1980s', '1990s', '2000s', '2010s']
seg5_median_income = ['0-30%','30-60%','60-80%']

community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg1_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg1_tenure) & 
                     community_results['build_existing_model.area_median_income'].isin(seg1_median_income),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg2_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg2_tenure) & 
                     community_results['build_existing_model.area_median_income'].isin(seg2_median_income),
                     'segment'] = 'segment2'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg3_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg3_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg3_tenure) & 
                     community_results['build_existing_model.area_median_income'].isin(seg3_median_income),
                     'segment'] = 'segment3'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg4_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg4_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg4_tenure) & 
                     community_results['build_existing_model.area_median_income'].isin(seg4_median_income),
                     'segment'] = 'segment4'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.vintage'].isin(seg5_vintage) &
                     community_results['build_existing_model.area_median_income'].isin(seg5_median_income),
                     'segment'] = 'segment5'

## 3.1 Upgrade Cost Per Dwelling Unit 

In [39]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]
saving_results[f'segment3_{income}_upgrade_cost_mean_usd'] = results[2]
saving_results[f'segment4_{income}_upgrade_cost_mean_usd'] = results[3]
saving_results[f'segment5_{income}_upgrade_cost_mean_usd'] = results[4]

segment1: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1127.3
Electric Clothes Dryer                                              1266.0
Induction Cooking                                                   1953.1
Heat Pump Clothes Dryer                                             2547.5
Heat Pump Water Heater                                              3014.7
Basic Enclosure                                                     4512.8
Enhanced Enclosure                                                  5123.9
High Efficiency Heat Pump with Electric Heat Backup                28860.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        32113.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     32113.6
High Efficiency Whole Home Electrification                         36375.5
Basic Enclosure + High Efficiency Whole Home Electrification       36614.1
Enhanced Enclosure + High Efficiency Whole Home Electrification    36614.1
Mininum Effi

segment2: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                     1127.3
Electric Clothes Dryer                                               1266.0
Induction Cooking                                                    1953.1
Heat Pump Clothes Dryer                                              2547.5
Heat Pump Water Heater                                               3150.2
Basic Enclosure                                                      4796.8
Enhanced Enclosure                                                   6372.4
High Efficiency Heat Pump with Electric Heat Backup                 42479.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         44412.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      44440.9
Basic Enclosure + High Efficiency Whole Home Electrification        48913.0
Enhanced Enclosure + High Efficiency Whole Home Electrification     48941.4
High Efficiency Whole Home Electrification                          50129.9

segment3: upgrade cost (usd) per dwelling unit


upgrade_name
Basic Enclosure                                                      864.8
Electric Cooking                                                    1127.3
Induction Cooking                                                   1953.1
Heat Pump Clothes Dryer                                             2547.5
Enhanced Enclosure                                                  2712.0
Heat Pump Water Heater                                              2905.1
High Efficiency Heat Pump with Electric Heat Backup                23127.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        25894.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     27632.5
Basic Enclosure + High Efficiency Whole Home Electrification       29659.5
High Efficiency Whole Home Electrification                         29798.1
Mininum Efficiency Heat Pump with Electric Heat Backup             30536.5
Mininum Efficiency Heat Pump with Existing Heat Backup             30536.5
Enhanced Enc

segment4: upgrade cost (usd) per dwelling unit


upgrade_name
Basic Enclosure                                                      605.9
Electric Cooking                                                    1127.3
Enhanced Enclosure                                                  1767.4
Induction Cooking                                                   1953.1
Heat Pump Clothes Dryer                                             2547.5
Heat Pump Water Heater                                              2905.1
High Efficiency Heat Pump with Electric Heat Backup                20270.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        21699.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     22217.9
Mininum Efficiency Whole Home Electrification                      22973.4
Basic Enclosure + High Efficiency Whole Home Electrification       24047.7
Enhanced Enclosure + High Efficiency Whole Home Electrification    24566.0
High Efficiency Whole Home Electrification                         25523.6
Mininum Effi

segment5: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1127.3
Electric Clothes Dryer                                              1266.0
Basic Enclosure                                                     1775.9
Induction Cooking                                                   1953.1
Enhanced Enclosure                                                  2391.5
Heat Pump Clothes Dryer                                             2547.5
Heat Pump Water Heater                                              3024.5
High Efficiency Heat Pump with Electric Heat Backup                23422.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        26846.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     27142.9
Mininum Efficiency Whole Home Electrification                      27579.0
High Efficiency Whole Home Electrification                         29820.5
Basic Enclosure + High Efficiency Whole Home Electrification       30220.3
Enhanced Enc

 ## 3.2 Annual Savings Per Dollar Invested

### Annual Bill Savings Per Dollar Invested

In [40]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_per_dollar_invested_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_per_dollar_invested_usd'] = results[3]
saving_results[f'segment5_{income}_saving_bill_per_dollar_invested_usd'] = results[4]

segment1


upgrade_name
Basic Enclosure                                                    0.07975
Enhanced Enclosure                                                 0.07652
Heat Pump Water Heater                                             0.04453
Heat Pump Clothes Dryer                                            0.00886
Basic Enclosure + High Efficiency Whole Home Electrification       0.00184
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00184
Induction Cooking                                                  0.00140
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00122
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00122
Mininum Efficiency Heat Pump with Existing Heat Backup            -0.00273
Electric Cooking                                                  -0.00533
Electric Clothes Dryer                                            -0.01531
High Efficiency Whole Home Electrification                        -0.01543
High Efficie

segment2


upgrade_name
Basic Enclosure                                                    0.07689
Enhanced Enclosure                                                 0.07153
Heat Pump Water Heater                                             0.02787
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.01138
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01078
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00986
Basic Enclosure + High Efficiency Whole Home Electrification       0.00940
Heat Pump Clothes Dryer                                            0.00696
Induction Cooking                                                  0.00047
Mininum Efficiency Heat Pump with Existing Heat Backup            -0.00040
High Efficiency Whole Home Electrification                        -0.00095
High Efficiency Heat Pump with Electric Heat Backup               -0.00527
Electric Cooking                                                  -0.00554
Mininum Effi

segment3


upgrade_name
Basic Enclosure                                                    0.24074
Enhanced Enclosure                                                 0.09782
Heat Pump Water Heater                                             0.02566
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02338
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02207
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02109
Basic Enclosure + High Efficiency Whole Home Electrification       0.01981
High Efficiency Whole Home Electrification                         0.01259
High Efficiency Heat Pump with Electric Heat Backup                0.00991
Heat Pump Clothes Dryer                                            0.00661
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00263
Induction Cooking                                                  0.00044
Mininum Efficiency Heat Pump with Electric Heat Backup            -0.00361
Mininum Effi

segment4


upgrade_name
Basic Enclosure                                                    0.27900
Enhanced Enclosure                                                 0.11279
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02320
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02318
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02126
Basic Enclosure + High Efficiency Whole Home Electrification       0.02120
High Efficiency Whole Home Electrification                         0.01746
High Efficiency Heat Pump with Electric Heat Backup                0.01732
Heat Pump Water Heater                                             0.01578
Heat Pump Clothes Dryer                                            0.00779
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00416
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00360
Mininum Efficiency Whole Home Electrification                      0.00042
Induction Co

segment5


upgrade_name
Basic Enclosure                                                    0.08603
Enhanced Enclosure                                                 0.07181
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.01422
Heat Pump Water Heater                                             0.01339
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.01313
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01309
Basic Enclosure + High Efficiency Whole Home Electrification       0.01211
High Efficiency Whole Home Electrification                         0.00865
Heat Pump Clothes Dryer                                            0.00717
High Efficiency Heat Pump with Electric Heat Backup                0.00606
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00122
Induction Cooking                                                  0.00041
Mininum Efficiency Heat Pump with Electric Heat Backup            -0.00405
Electric Coo

 ### Annual Energy Savings Per Dollar Invested

In [41]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[3]
saving_results[f'segment5_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[4]

segment1


upgrade_name
Basic Enclosure                                                    0.01281
Enhanced Enclosure                                                 0.01192
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00387
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00387
Basic Enclosure + High Efficiency Whole Home Electrification       0.00344
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00344
High Efficiency Heat Pump with Electric Heat Backup                0.00288
Heat Pump Water Heater                                             0.00282
High Efficiency Whole Home Electrification                         0.00276
Mininum Efficiency Whole Home Electrification                      0.00166
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00145
Electric Cooking                                                   0.00138
Heat Pump Clothes Dryer                                            0.00047
Electric Clo

segment2


upgrade_name
Basic Enclosure                                                    0.01008
Enhanced Enclosure                                                 0.00867
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00331
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00325
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00304
Basic Enclosure + High Efficiency Whole Home Electrification       0.00298
High Efficiency Heat Pump with Electric Heat Backup                0.00263
High Efficiency Whole Home Electrification                         0.00250
Heat Pump Water Heater                                             0.00249
Electric Cooking                                                   0.00122
Mininum Efficiency Whole Home Electrification                      0.00071
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00061
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00046
Electric Clo

segment3


...

segment4


upgrade_name
Basic Enclosure                                                    0.01468
Enhanced Enclosure                                                 0.00610
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00139
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00138
Heat Pump Water Heater                                             0.00131
Basic Enclosure + High Efficiency Whole Home Electrification       0.00129
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00128
Electric Cooking                                                   0.00118
High Efficiency Whole Home Electrification                         0.00111
High Efficiency Heat Pump with Electric Heat Backup                0.00104
Mininum Efficiency Whole Home Electrification                      0.00070
Heat Pump Clothes Dryer                                            0.00032
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00028
Induction Co

segment5


upgrade_name
Basic Enclosure                                                    0.00744
Enhanced Enclosure                                                 0.00609
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00196
Heat Pump Water Heater                                             0.00194
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00193
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00178
Basic Enclosure + High Efficiency Whole Home Electrification       0.00175
High Efficiency Whole Home Electrification                         0.00163
High Efficiency Heat Pump with Electric Heat Backup                0.00156
Electric Cooking                                                   0.00140
Mininum Efficiency Whole Home Electrification                      0.00129
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00053
Electric Clothes Dryer                                             0.00052
Heat Pump Cl

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/2996368806.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/2996368806.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/2996368806.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

 ### Annual Emission Savings Per Dollar Invested

In [42]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[3]
saving_results[f'segment5_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[4]

segment1


upgrade_name
Basic Enclosure                                                    0.95085
Enhanced Enclosure                                                 0.87947
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.32988
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.32988
High Efficiency Heat Pump with Electric Heat Backup                0.29868
Basic Enclosure + High Efficiency Whole Home Electrification       0.29163
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.29163
High Efficiency Whole Home Electrification                         0.26683
Mininum Efficiency Whole Home Electrification                      0.20412
Mininum Efficiency Heat Pump with Electric Heat Backup             0.19470
Heat Pump Water Heater                                             0.14440
Electric Cooking                                                   0.13197
Electric Clothes Dryer                                             0.06878
Mininum Effi

segment2


upgrade_name
Basic Enclosure                                                    0.71415
Enhanced Enclosure                                                 0.60316
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.25934
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.25712
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.23732
Basic Enclosure + High Efficiency Whole Home Electrification       0.23530
High Efficiency Heat Pump with Electric Heat Backup                0.23264
High Efficiency Whole Home Electrification                         0.21370
Heat Pump Water Heater                                             0.15542
Electric Cooking                                                   0.11840
Mininum Efficiency Whole Home Electrification                      0.07686
Electric Clothes Dryer                                             0.06918
Mininum Efficiency Heat Pump with Electric Heat Backup             0.06606
Mininum Effi

segment3


...

segment4


upgrade_name
Basic Enclosure                                                    0.70726
Enhanced Enclosure                                                 0.29995
Electric Cooking                                                   0.12138
Heat Pump Water Heater                                             0.08111
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.07355
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.07269
Basic Enclosure + High Efficiency Whole Home Electrification       0.06835
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.06769
High Efficiency Whole Home Electrification                         0.06047
Mininum Efficiency Whole Home Electrification                      0.05970
High Efficiency Heat Pump with Electric Heat Backup                0.05457
Induction Cooking                                                  0.02379
Mininum Efficiency Heat Pump with Electric Heat Backup             0.01690
Mininum Effi

segment5


upgrade_name
Basic Enclosure                                                    0.46427
Enhanced Enclosure                                                 0.37794
Heat Pump Water Heater                                             0.14015
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.13796
Electric Cooking                                                   0.13777
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.13751
Mininum Efficiency Whole Home Electrification                      0.13239
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.12523
Basic Enclosure + High Efficiency Whole Home Electrification       0.12471
High Efficiency Whole Home Electrification                         0.12071
High Efficiency Heat Pump with Electric Heat Backup                0.11904
Electric Clothes Dryer                                             0.09415
Mininum Efficiency Heat Pump with Electric Heat Backup             0.05172
Induction Co

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/3746797145.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/3746797145.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/3746797145.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

## 3.3 Annual Total Savings

### Annual Total Bill Savings

In [43]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_total_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_total_usd'] = results[3]
saving_results[f'segment5_{income}_saving_bill_total_usd'] = results[4]

segment1


upgrade_name
Enhanced Enclosure                                                 127060.8
Basic Enclosure                                                    116635.7
Heat Pump Water Heater                                              43508.3
Basic Enclosure + High Efficiency Whole Home Electrification        21820.5
Enhanced Enclosure + High Efficiency Whole Home Electrification     21820.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         12747.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      12747.7
Heat Pump Clothes Dryer                                              7310.7
Induction Cooking                                                     884.9
Electric Cooking                                                     -322.3
Electric Clothes Dryer                                              -1371.2
Mininum Efficiency Heat Pump with Existing Heat Backup             -34822.4
High Efficiency Whole Home Electrification                        -181912.1

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification     512523.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      491290.9
Basic Enclosure + High Efficiency Whole Home Electrification        446915.6
Enhanced Enclosure                                                  442900.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         425682.9
Basic Enclosure                                                     344289.5
Heat Pump Water Heater                                               85299.9
Heat Pump Clothes Dryer                                              17219.4
Induction Cooking                                                      887.0
Electric Cooking                                                     -2030.1
Electric Clothes Dryer                                               -3186.7
Mininum Efficiency Heat Pump with Existing Heat Backup              -30679.6
High Efficiency Whole Home Electrification                     

segment3


...

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    990883.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     977988.8
Basic Enclosure + High Efficiency Whole Home Electrification       967299.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        954404.1
High Efficiency Whole Home Electrification                         845441.3
High Efficiency Heat Pump with Electric Heat Backup                666194.5
Mininum Efficiency Heat Pump with Existing Heat Backup             381660.7
Mininum Efficiency Heat Pump with Electric Heat Backup             339689.4
Enhanced Enclosure                                                 306733.0
Basic Enclosure                                                    250993.3
Heat Pump Water Heater                                              86959.0
Mininum Efficiency Whole Home Electrification                       11695.2
Heat Pump Clothes Dryer                                              9127.6

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1868827.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1806337.5
Basic Enclosure + High Efficiency Whole Home Electrification       1711631.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1649141.5
High Efficiency Whole Home Electrification                         1206344.1
High Efficiency Heat Pump with Electric Heat Backup                 664565.9
Enhanced Enclosure                                                  635501.8
Basic Enclosure                                                     533354.5
Mininum Efficiency Heat Pump with Existing Heat Backup              230113.6
Heat Pump Water Heater                                              189471.3
Heat Pump Clothes Dryer                                              47628.4
Induction Cooking                                                     3766.1
Electric Cooking                                               

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/320912423.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/320912423.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/320912423.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

### Annual Total Energy Savings

In [44]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_total_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_total_mmbtu'] = results[3]
saving_results[f'segment5_{income}_saving_energy_total_mmbtu'] = results[4]

segment1


upgrade_name
Basic Enclosure + High Efficiency Whole Home Electrification       40795.3
Enhanced Enclosure + High Efficiency Whole Home Electrification    40795.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        40276.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     40276.0
High Efficiency Whole Home Electrification                         32503.8
High Efficiency Heat Pump with Electric Heat Backup                26948.2
Mininum Efficiency Whole Home Electrification                      22714.8
Enhanced Enclosure                                                 19790.1
Basic Enclosure                                                    18730.6
Mininum Efficiency Heat Pump with Electric Heat Backup             18498.4
Mininum Efficiency Heat Pump with Existing Heat Backup              3035.5
Heat Pump Water Heater                                              2750.5
Heat Pump Clothes Dryer                                              386.9
Induction Co

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    144470.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     143004.5
Basic Enclosure + High Efficiency Whole Home Electrification       141800.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        140334.4
High Efficiency Whole Home Electrification                         121727.9
High Efficiency Heat Pump with Electric Heat Backup                108351.8
Mininum Efficiency Whole Home Electrification                       82541.4
Mininum Efficiency Heat Pump with Electric Heat Backup              77811.4
Enhanced Enclosure                                                  53704.3
Basic Enclosure                                                     45130.0
Mininum Efficiency Heat Pump with Existing Heat Backup              35931.1
Heat Pump Water Heater                                               7622.1
Heat Pump Clothes Dryer                                               899.3

segment3


...

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    59600.4
Basic Enclosure + High Efficiency Whole Home Electrification       58640.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     58318.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        57358.3
High Efficiency Whole Home Electrification                         53681.2
High Efficiency Heat Pump with Electric Heat Backup                39857.7
Mininum Efficiency Heat Pump with Electric Heat Backup             26572.3
Mininum Efficiency Heat Pump with Existing Heat Backup             24301.3
Mininum Efficiency Whole Home Electrification                      19749.2
Enhanced Enclosure                                                 16578.2
Basic Enclosure                                                    13202.1
Heat Pump Water Heater                                              7227.4
Induction Cooking                                                    910.8
Electric Coo

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    254242.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     248792.7
Basic Enclosure + High Efficiency Whole Home Electrification       247845.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        242395.1
High Efficiency Whole Home Electrification                         227281.1
High Efficiency Heat Pump with Electric Heat Backup                170687.6
Mininum Efficiency Whole Home Electrification                      128539.6
Mininum Efficiency Heat Pump with Electric Heat Backup             109485.9
Enhanced Enclosure                                                  53911.2
Basic Enclosure                                                     46103.7
Mininum Efficiency Heat Pump with Existing Heat Backup              35309.7
Heat Pump Water Heater                                              27521.9
Heat Pump Clothes Dryer                                              2735.9

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/4074720020.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/4074720020.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/4074720020.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

### Annual Total Emission Savings

In [45]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_total_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_total_kgCO2e'] = results[3]
saving_results[f'segment5_{income}_saving_emission_total_kgCO2e'] = results[4]

segment1


upgrade_name
Basic Enclosure + High Efficiency Whole Home Electrification       3460473.7
Enhanced Enclosure + High Efficiency Whole Home Electrification    3460473.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3433300.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3433300.3
High Efficiency Whole Home Electrification                         3145572.2
High Efficiency Heat Pump with Electric Heat Backup                2793601.9
Mininum Efficiency Whole Home Electrification                      2793545.1
Mininum Efficiency Heat Pump with Electric Heat Backup             2485842.5
Enhanced Enclosure                                                 1460444.3
Basic Enclosure                                                    1390657.7
Mininum Efficiency Heat Pump with Existing Heat Backup              333453.9
Heat Pump Water Heater                                              141084.6
Heat Pump Clothes Dryer                                        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    11285637.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     11198505.5
Basic Enclosure + High Efficiency Whole Home Electrification       11182798.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        11095666.5
High Efficiency Whole Home Electrification                         10409065.4
High Efficiency Heat Pump with Electric Heat Backup                 9602368.7
Mininum Efficiency Whole Home Electrification                       8971931.0
Mininum Efficiency Heat Pump with Electric Heat Backup              8469300.7
Enhanced Enclosure                                                  3734647.5
Basic Enclosure                                                     3197900.4
Mininum Efficiency Heat Pump with Existing Heat Backup              3130076.7
Heat Pump Water Heater                                               475740.1
Induction Cooking                                  

segment3


...

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    3154664.5
Basic Enclosure + High Efficiency Whole Home Electrification       3118411.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3064109.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3027855.8
High Efficiency Whole Home Electrification                         2928056.0
High Efficiency Heat Pump with Electric Heat Backup                2098809.7
Mininum Efficiency Whole Home Electrification                      1672433.1
Mininum Efficiency Heat Pump with Electric Heat Backup             1595898.3
Mininum Efficiency Heat Pump with Existing Heat Backup             1326731.1
Enhanced Enclosure                                                  815709.7
Basic Enclosure                                                     636263.5
Heat Pump Water Heater                                              447039.7
Induction Cooking                                              

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    17879805.0
Basic Enclosure + High Efficiency Whole Home Electrification       17632724.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     17519412.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        17272331.7
High Efficiency Whole Home Electrification                         16841317.4
Mininum Efficiency Whole Home Electrification                      13189362.3
High Efficiency Heat Pump with Electric Heat Backup                13045008.6
Mininum Efficiency Heat Pump with Electric Heat Backup             10729909.3
Enhanced Enclosure                                                  3344590.2
Basic Enclosure                                                     2878159.2
Mininum Efficiency Heat Pump with Existing Heat Backup              2569902.3
Heat Pump Water Heater                                              1983217.9
Induction Cooking                                  

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/723531023.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/723531023.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/723531023.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

## 3.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [46]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_mean_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_mean_usd'] = results[3]
saving_results[f'segment5_{income}_saving_bill_mean_usd'] = results[4]

segment1


upgrade_name
Enhanced Enclosure                                                  392.1
Basic Enclosure                                                     359.9
Heat Pump Water Heater                                              134.2
Basic Enclosure + High Efficiency Whole Home Electrification         67.3
Enhanced Enclosure + High Efficiency Whole Home Electrification      67.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup          39.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup       39.3
Heat Pump Clothes Dryer                                              22.6
Induction Cooking                                                     2.7
Electric Cooking                                                     -6.0
Electric Clothes Dryer                                              -19.4
Mininum Efficiency Heat Pump with Existing Heat Backup             -107.4
High Efficiency Whole Home Electrification                         -561.3
High Efficiency Heat Pump

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification     527.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      505.6
Basic Enclosure + High Efficiency Whole Home Electrification        460.0
Enhanced Enclosure                                                  455.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         438.1
Basic Enclosure                                                     368.8
Heat Pump Water Heater                                               87.8
Heat Pump Clothes Dryer                                              17.7
Induction Cooking                                                     0.9
Electric Cooking                                                     -6.2
Electric Clothes Dryer                                              -21.9
Mininum Efficiency Heat Pump with Existing Heat Backup              -34.9
High Efficiency Whole Home Electrification                          -47.6
High Efficiency Heat Pump

segment3


...

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    522.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     515.5
Basic Enclosure + High Efficiency Whole Home Electrification       509.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        503.0
High Efficiency Whole Home Electrification                         445.6
High Efficiency Heat Pump with Electric Heat Backup                351.1
Mininum Efficiency Heat Pump with Existing Heat Backup             227.0
Enhanced Enclosure                                                 199.3
Mininum Efficiency Heat Pump with Electric Heat Backup             179.0
Basic Enclosure                                                    169.1
Heat Pump Water Heater                                              45.8
Heat Pump Clothes Dryer                                             19.8
Mininum Efficiency Whole Home Electrification                        9.6
Induction Cooking                     

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    399.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     386.1
Basic Enclosure + High Efficiency Whole Home Electrification       365.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        352.5
High Efficiency Whole Home Electrification                         257.8
Enhanced Enclosure                                                 171.7
Basic Enclosure                                                    152.8
High Efficiency Heat Pump with Electric Heat Backup                142.0
Mininum Efficiency Heat Pump with Existing Heat Backup              49.7
Heat Pump Water Heater                                              40.5
Heat Pump Clothes Dryer                                             18.3
Induction Cooking                                                    0.8
Electric Cooking                                                    -8.0
Electric Clothes Dryer                

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/541036721.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/541036721.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/541036721.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

### Annual Energy Savings Per Dwelling Unit

In [47]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_mean_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_mean_mmbtu'] = results[3]
saving_results[f'segment5_{income}_saving_energy_mean_mmbtu'] = results[4]

segment1


upgrade_name
Basic Enclosure + High Efficiency Whole Home Electrification       125.9
Enhanced Enclosure + High Efficiency Whole Home Electrification    125.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        124.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     124.3
High Efficiency Whole Home Electrification                         100.3
High Efficiency Heat Pump with Electric Heat Backup                 83.2
Mininum Efficiency Whole Home Electrification                       70.1
Enhanced Enclosure                                                  61.1
Basic Enclosure                                                     57.8
Mininum Efficiency Heat Pump with Electric Heat Backup              57.1
Mininum Efficiency Heat Pump with Existing Heat Backup               9.4
Heat Pump Water Heater                                               8.5
Electric Cooking                                                     1.6
Heat Pump Clothes Dryer               

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    148.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     147.2
Basic Enclosure + High Efficiency Whole Home Electrification       145.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        144.4
High Efficiency Whole Home Electrification                         125.3
High Efficiency Heat Pump with Electric Heat Backup                111.5
Mininum Efficiency Whole Home Electrification                       93.8
Mininum Efficiency Heat Pump with Electric Heat Backup              80.1
Enhanced Enclosure                                                  55.3
Basic Enclosure                                                     48.3
Mininum Efficiency Heat Pump with Existing Heat Backup              40.9
Heat Pump Water Heater                                               7.8
Electric Cooking                                                     1.4
Heat Pump Clothes Dryer               

segment3


...

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    31.4
Basic Enclosure + High Efficiency Whole Home Electrification       30.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     30.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        30.2
High Efficiency Whole Home Electrification                         28.3
High Efficiency Heat Pump with Electric Heat Backup                21.0
Mininum Efficiency Whole Home Electrification                      16.2
Mininum Efficiency Heat Pump with Existing Heat Backup             14.5
Mininum Efficiency Heat Pump with Electric Heat Backup             14.0
Enhanced Enclosure                                                 10.8
Basic Enclosure                                                     8.9
Heat Pump Water Heater                                              3.8
Electric Cooking                                                    1.3
Heat Pump Clothes Dryer                            

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    54.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     53.2
Basic Enclosure + High Efficiency Whole Home Electrification       53.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        51.8
High Efficiency Whole Home Electrification                         48.6
High Efficiency Heat Pump with Electric Heat Backup                36.5
Mininum Efficiency Whole Home Electrification                      35.6
Mininum Efficiency Heat Pump with Electric Heat Backup             23.4
Enhanced Enclosure                                                 14.6
Basic Enclosure                                                    13.2
Mininum Efficiency Heat Pump with Existing Heat Backup              7.6
Heat Pump Water Heater                                              5.9
Electric Cooking                                                    1.6
Heat Pump Clothes Dryer                            

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/927876618.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/927876618.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/927876618.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

### Annual Percentage Energy Savings Per Dwelling Unit

In [48]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]
saving_results[f'segment3_{income}_pct_saving_energy_mean_%'] = results[2]
saving_results[f'segment4_{income}_pct_saving_energy_mean_%'] = results[3]
saving_results[f'segment5_{income}_pct_saving_energy_mean_%'] = results[4]

segment1


upgrade_name
Basic Enclosure + High Efficiency Whole Home Electrification       61.7
Enhanced Enclosure + High Efficiency Whole Home Electrification    61.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        60.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     60.8
High Efficiency Whole Home Electrification                         48.5
High Efficiency Heat Pump with Electric Heat Backup                38.8
Mininum Efficiency Whole Home Electrification                      33.7
Enhanced Enclosure                                                 29.4
Basic Enclosure                                                    26.8
Mininum Efficiency Heat Pump with Electric Heat Backup             26.0
Heat Pump Water Heater                                              5.4
Mininum Efficiency Heat Pump with Existing Heat Backup              3.0
Electric Cooking                                                    0.9
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    67.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     66.7
Basic Enclosure + High Efficiency Whole Home Electrification       66.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        65.9
High Efficiency Whole Home Electrification                         57.8
High Efficiency Heat Pump with Electric Heat Backup                50.4
Mininum Efficiency Whole Home Electrification                      42.7
Mininum Efficiency Heat Pump with Electric Heat Backup             36.1
Enhanced Enclosure                                                 24.7
Basic Enclosure                                                    20.7
Mininum Efficiency Heat Pump with Existing Heat Backup             20.5
Heat Pump Water Heater                                              4.2
Electric Cooking                                                    0.8
Heat Pump Clothes Dryer                            

segment3


...

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    54.7
Basic Enclosure + High Efficiency Whole Home Electrification       53.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     53.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        52.6
High Efficiency Whole Home Electrification                         49.7
High Efficiency Heat Pump with Electric Heat Backup                34.8
Mininum Efficiency Whole Home Electrification                      25.0
Mininum Efficiency Heat Pump with Existing Heat Backup             23.8
Mininum Efficiency Heat Pump with Electric Heat Backup             21.8
Enhanced Enclosure                                                 17.3
Basic Enclosure                                                    13.8
Heat Pump Water Heater                                              8.8
Electric Cooking                                                    2.4
Heat Pump Clothes Dryer                            

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    57.3
Basic Enclosure + High Efficiency Whole Home Electrification       56.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     56.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        54.9
High Efficiency Whole Home Electrification                         52.0
High Efficiency Heat Pump with Electric Heat Backup                36.5
Mininum Efficiency Whole Home Electrification                      35.0
Mininum Efficiency Heat Pump with Electric Heat Backup             23.0
Enhanced Enclosure                                                 13.3
Basic Enclosure                                                    11.6
Mininum Efficiency Heat Pump with Existing Heat Backup             10.1
Heat Pump Water Heater                                              7.6
Electric Cooking                                                    1.6
Heat Pump Clothes Dryer                            

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/1754187417.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/1754187417.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/1754187417.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

### Annual Emission Savings Per Dwelling Unit

In [49]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_mean_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_mean_kgCO2e'] = results[3]
saving_results[f'segment5_{income}_saving_emission_mean_kgCO2e'] = results[4]

segment1


upgrade_name
Basic Enclosure + High Efficiency Whole Home Electrification       10677.6
Enhanced Enclosure + High Efficiency Whole Home Electrification    10677.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        10593.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     10593.8
High Efficiency Whole Home Electrification                          9706.0
High Efficiency Heat Pump with Electric Heat Backup                 8619.9
Mininum Efficiency Whole Home Electrification                       8619.7
Mininum Efficiency Heat Pump with Electric Heat Backup              7670.3
Enhanced Enclosure                                                  4506.3
Basic Enclosure                                                     4291.0
Mininum Efficiency Heat Pump with Existing Heat Backup              1028.9
Heat Pump Water Heater                                               435.3
Electric Cooking                                                     148.8
Electric Clo

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    11614.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     11525.2
Basic Enclosure + High Efficiency Whole Home Electrification       11509.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        11419.4
High Efficiency Whole Home Electrification                         10712.8
Mininum Efficiency Whole Home Electrification                      10194.8
High Efficiency Heat Pump with Electric Heat Backup                 9882.5
Mininum Efficiency Heat Pump with Electric Heat Backup              8716.4
Enhanced Enclosure                                                  3843.6
Mininum Efficiency Heat Pump with Existing Heat Backup              3562.7
Basic Enclosure                                                     3425.7
Heat Pump Water Heater                                               489.6
Electric Cooking                                                     133.5
Electric Clo

segment3


...

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1662.8
Basic Enclosure + High Efficiency Whole Home Electrification       1643.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1615.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1595.9
High Efficiency Whole Home Electrification                         1543.3
Mininum Efficiency Whole Home Electrification                      1371.4
High Efficiency Heat Pump with Electric Heat Backup                1106.2
Mininum Efficiency Heat Pump with Electric Heat Backup              841.2
Mininum Efficiency Heat Pump with Existing Heat Backup              789.2
Enhanced Enclosure                                                  530.1
Basic Enclosure                                                     428.5
Heat Pump Water Heater                                              235.6
Electric Cooking                                                    136.8
Induction Cooking        

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    3821.6
Basic Enclosure + High Efficiency Whole Home Electrification       3768.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3744.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3691.7
Mininum Efficiency Whole Home Electrification                      3651.3
High Efficiency Whole Home Electrification                         3599.6
High Efficiency Heat Pump with Electric Heat Backup                2788.2
Mininum Efficiency Heat Pump with Electric Heat Backup             2293.4
Enhanced Enclosure                                                  903.8
Basic Enclosure                                                     824.5
Mininum Efficiency Heat Pump with Existing Heat Backup              555.5
Heat Pump Water Heater                                              423.9
Electric Cooking                                                    155.3
Electric Clothes Dryer   

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/3614653602.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/3614653602.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/3614653602.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

## 3.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [50]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/868763001.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/868763001.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/868763001.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

### Segment 2

In [51]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/3209730133.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/3209730133.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/3209730133.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

### Segment 3

In [52]:
egment = 'segment3'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment3_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment3_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment3_{income}_pct_energy_positive_savings_%'] = results[2]*100

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/1326448381.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/1326448381.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/1326448381.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

### Segment 4

In [53]:
segment = 'segment4'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment4_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment4_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment4_{income}_pct_energy_positive_savings_%'] = results[2]*100

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/2289506022.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/2289506022.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/2289506022.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

### Segment 5

In [54]:
segment = 'segment5'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment5_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment5_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment5_{income}_pct_energy_positive_savings_%'] = results[2]*100

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/1874328925.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/1874328925.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/1874328925.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

## 3.6. Average Energy Burden (%) Before and after Upgrade

In [55]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]
saving_results[[f'segment3_{income}_energy_burden_before_upgrade_%',f'segment3_{income}_energy_burden_after_upgrade_%']] = results[2]
saving_results[[f'segment4_{income}_energy_burden_before_upgrade_%',f'segment4_{income}_energy_burden_after_upgrade_%']] = results[3]
saving_results[[f'segment5_{income}_energy_burden_before_upgrade_%',f'segment5_{income}_energy_burden_after_upgrade_%']] = results[4]

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/2591935474.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/2591935474.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/2591935474.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

energy burden (%) before and after upgrade
segment1


,before,after
upgrade_name,,
Enhanced Enclosure,18.1,13.9
Basic Enclosure,18.1,14.2
Basic Enclosure + High Efficiency Whole Home Electrification,18.1,17.8
Enhanced Enclosure + High Efficiency Whole Home Electrification,18.1,17.8
Heat Pump Water Heater,18.1,17.8
Heat Pump Clothes Dryer,18.1,17.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,18.1,18.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,18.1,18.0
Induction Cooking,18.1,18.1


segment2


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,15.4,10.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,15.4,10.2
Basic Enclosure + High Efficiency Whole Home Electrification,15.4,10.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,15.4,10.7
Enhanced Enclosure,15.4,12.8
High Efficiency Whole Home Electrification,15.4,12.9
Basic Enclosure,15.9,13.9
High Efficiency Heat Pump with Electric Heat Backup,15.4,13.9
Mininum Efficiency Heat Pump with Existing Heat Backup,14.8,14.2


...

segment4


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,8.7,4.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,8.7,4.5
Basic Enclosure + High Efficiency Whole Home Electrification,8.7,4.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,8.7,4.7
High Efficiency Whole Home Electrification,8.7,5.3
High Efficiency Heat Pump with Electric Heat Backup,8.7,6.0
Mininum Efficiency Heat Pump with Existing Heat Backup,9.7,8.0
Enhanced Enclosure,9.6,8.0
Basic Enclosure,9.8,8.5


segment5


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,8.2,5.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,8.2,5.8
Basic Enclosure + High Efficiency Whole Home Electrification,8.2,5.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,8.2,5.9
High Efficiency Whole Home Electrification,8.2,6.5
High Efficiency Heat Pump with Electric Heat Backup,8.2,7.1
Enhanced Enclosure,7.9,7.0
Basic Enclosure,7.9,7.1
Mininum Efficiency Heat Pump with Existing Heat Backup,8.1,7.7


/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/3291905064.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/3291905064.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_25540/3291905064.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

In [56]:
saving_results.to_csv(fig_dir / f'saving_results_{community}.csv')

### Save html file

In [57]:
! jupyter nbconvert --output-dir='./results/duluth/' --to=html --no-input saving_results_reports_for_duluth.ipynb

[NbConvertApp] Converting notebook saving_results_reports_for_duluth.ipynb to html
/Users/lliu2/micromamba/envs/res-est/share/jupyter/nbconvert/templates/base/display_priority.j2:32: UserWarning: Your element with mimetype(s) dict_keys(['application/vnd.plotly.v1+json']) is not able to be represented.
  {%- elif type == 'application/javascript' -%}
[NbConvertApp] Writing 557861 bytes to results/duluth/saving_results_reports_for_duluth.html
